In [627]:
from experiments.datasets import *
import torch
from os import listdir

In [626]:
print("Loading user graph")
g = load_ig_graph()

fname = "../../../centralities.pickle"
print("Loading pre-computed centralities")
with open(fname, 'rb') as f:
    centralities = pickle.load(f)


Loading user graph
Loading pre-computed centralities


In [648]:
# DATAFRAMES_PATH = "/users/pcelayes/repos/sna_classifier/data/dataframes"
from os.path import join
import pandas as pd


train_samples = {}
test_samples = {}


for fname in listdir(DATAFRAMES_FOLDER):
    df_path = join(DATAFRAMES_FOLDER, fname)
    if fname.startswith("dfXtrain__fix_new_hps_"):
        user_id = fname.split(".")[0].split("_")[-1]
        if not user_id:
            continue
        Xy_train = pd.read_pickle(df_path)
        train_samples[user_id] = Xy_train.index.values.tolist()
    elif fname.startswith("dfXtest__fix_new_hps_"):
        if not user_id:
            continue
        user_id = fname.split(".")[0].split("_")[-1]
        Xy_test = pd.read_pickle(df_path)
        test_samples[user_id] = Xy_test.index.values.tolist()        

In [649]:
len(train_samples)

4687

In [650]:
len(test_samples)

4687

In [560]:
Xy_train.sum(axis=1).sort_values()

636204372133834753     1.0
639125746540265472     1.0
642320495464062980     1.0
639521192517484544     1.0
642093328205545472     1.0
                      ... 
643491104478859264     3.0
639542555147902976     3.0
642803633164124160     3.0
378360917245067264     4.0
641415151573430272    11.0
Length: 3500, dtype: float64

First step: encode a (user, tuit) example a as a Data example for pyg
- One-hot encoded node ids
- flags for: central user (u), neighbors, t-active neighbors
- Pass all embeddings through the same embedding layer + GCN layers
- aggregation step:
  - N: agreggate neighbors
  - A: aggregate active neighbors
  - concatenate u + A + N
  - pass through MLP layer

In [561]:
user_id

362040004

In [562]:
tweet_id = 641415151573430272

In [563]:
s = open_session()
user = s.query(User).get(user_id)
neighbours = get_level2_neighbours(user, s)

# remove central user from neighbours
neighbours = [u for u in neighbours if u.id != user.id]
neighbour_ids = [u.id for u in neighbours]

In [564]:
len(neighbours)

616

In [565]:
retweeting_neighbours = []
other_neighbours = []
for u in neighbours:
    if tweet_id in [t.id for t in u.timeline]:
        retweeting_neighbours.append(u)
    else:
        other_neighbours.append(u)

In [566]:
len(retweeting_neighbours)

11

In [567]:
len(other_neighbours)

605

In [568]:
from torch_geometric.data import Data

In [569]:
Data?

For now, I will just pass the full graph and edges with each example, only changing the last features:
 - is central
 - is neighbor
 - is retweeting

TODO: figure out how to pass only the nodes in the neighborhood for each example

- Idea 1: pass already one-hot encoded
- Idea 2: learn if `pos` parameter can be used for this

Crear índice para one-hot encoding de nodos

In [570]:
def get_onehot(user_id):
    v = np.zeros(len(g.vs))
    ind = g.vs['twid'].index(str(user_id))
    v[ind] = 1
    return v

In [571]:
get_onehot(neighbour_ids[20]).sum()

1.0

In [572]:
# # x: features de nodos (u central + RT neighbours + others)
# features = [get_onehot(user_id) + ]

# # edge_index: conexiones entre ellos


# d = Data(x=x, edge_index=edge_index)

### Implementación más sencilla

- sin embeddings
- sólo el vecindario de u
- features: is_u, is_rt, centralidades (misma info que paper anterior)

In [573]:
index_users = [user] + neighbours
index_feats = [user_id] + neighbour_ids

In [574]:
index_feats

[362040004,
 154294345,
 114510872,
 28553977,
 15206492,
 7598672,
 150319476,
 149096668,
 7409742,
 156683723,
 61597563,
 269401801,
 80857373,
 145424446,
 68776651,
 393708005,
 201530028,
 14231827,
 7881912,
 24343505,
 29111965,
 312741533,
 35772042,
 162103727,
 156047129,
 139160485,
 92307605,
 139953366,
 267945256,
 139535901,
 404247722,
 136349459,
 145677457,
 163652140,
 103614732,
 144171947,
 218894949,
 184892265,
 93124373,
 154785800,
 175923801,
 11103932,
 35778170,
 99566858,
 793119246,
 82004804,
 150638214,
 229457613,
 2492299893,
 236622401,
 17429002,
 282654132,
 330557438,
 147647383,
 182925466,
 171616155,
 51480977,
 181408748,
 31423048,
 159851000,
 162171015,
 102407528,
 7806312,
 1433204424,
 50521735,
 16876964,
 70557301,
 12894042,
 66521679,
 17946159,
 159248716,
 7995672,
 108968018,
 133950381,
 274144952,
 160609570,
 115140176,
 159480540,
 123340910,
 150372271,
 172458477,
 24735713,
 174027585,
 69555752,
 613508372,
 143852870,
 1

In [575]:
g.vs["twid"]

['51864127',
 '43368531',
 '163143185',
 '11545',
 '143562981',
 '147527932',
 '190844485',
 '49731619',
 '50750455',
 '23022690',
 '271507198',
 '14519302',
 '46101166',
 '52816889',
 '150123282',
 '8732352',
 '178166167',
 '163884594',
 '94156238',
 '36049904',
 '8150842',
 '75814261',
 '63155327',
 '27511431',
 '243243040',
 '434933876',
 '51480977',
 '149206430',
 '185783907',
 '6825792',
 '972651',
 '355694505',
 '59011891',
 '17898894',
 '252633165',
 '77046009',
 '214642493',
 '218659553',
 '15866188',
 '17576586',
 '2003581',
 '151946454',
 '101958846',
 '37677874',
 '383257612',
 '149657401',
 '173348160',
 '557743160',
 '127709439',
 '158419035',
 '4010871',
 '132967105',
 '50752773',
 '153133506',
 '173268786',
 '19084628',
 '7885412',
 '2897441',
 '142476466',
 '75178495',
 '54100908',
 '116491488',
 '92656579',
 '122723516',
 '155216270',
 '3120871',
 '395083976',
 '146686284',
 '15355812',
 '91609020',
 '19886419',
 '34016889',
 '69338201',
 '14567622',
 '109382880',
 '12

In [576]:
# print("# Filter centralities to cover only ngids")
index_feats_inds_map = {int(l): i for (i,l) in enumerate(g.vs["twid"]) if int(l) in index_feats}
index_feats_inds = [ng_inds_map[l] for l in index_feats]
index_feats_centralities = [np.array(m)[ng_inds] for m in centralities]

In [577]:
index_feats_centralities[0].shape

(617,)

In [578]:
type(centralities[0])

list

In [579]:
centralities_matrix = np.vstack(index_feats_centralities).transpose()
centralities_matrix.shape

(617, 5)

In [580]:
is_central_col = np.zeros(len(index_feats))
is_central_col[0] = 1
is_central_col = np.expand_dims(is_central_col, axis=1)
is_central_col.shape

(617, 1)

In [581]:
is_retweeting_col = np.zeros(len(index_feats))
for i, u in enumerate(index_users):
    if tweet_id in [t.id for t in u.timeline]:
        is_retweeting_col[i] = 1
is_retweeting_col.sum()

11.0

In [582]:
is_retweeting_col = np.expand_dims(is_retweeting_col, axis=1)
is_retweeting_col.shape

(617, 1)

In [583]:
x = np.hstack([centralities_matrix, is_central_col, is_retweeting_col])
x.shape

(617, 7)

In [584]:
g.es

In [585]:
type(index_feats_inds)

list

In [586]:
index_feats_inds

[3678,
 1467,
 1970,
 3475,
 1343,
 5492,
 3872,
 2433,
 3283,
 2350,
 3920,
 1376,
 4118,
 5246,
 5431,
 3403,
 5025,
 4734,
 3284,
 1332,
 4677,
 2643,
 3229,
 1724,
 1917,
 4709,
 4240,
 2891,
 3157,
 5272,
 2575,
 740,
 215,
 5430,
 2653,
 612,
 5177,
 2906,
 2157,
 5127,
 4375,
 1109,
 1543,
 1825,
 5114,
 1464,
 4685,
 864,
 2674,
 2863,
 4655,
 3106,
 1887,
 2450,
 5550,
 4282,
 26,
 3902,
 4366,
 3166,
 4007,
 762,
 5065,
 1655,
 1866,
 4578,
 503,
 655,
 1595,
 4638,
 5353,
 5068,
 4443,
 4426,
 3472,
 1872,
 2753,
 3219,
 3177,
 4297,
 980,
 4885,
 777,
 3650,
 2289,
 4583,
 5105,
 617,
 1974,
 639,
 4397,
 2269,
 897,
 2387,
 5039,
 232,
 590,
 4058,
 3406,
 564,
 2968,
 343,
 2845,
 679,
 500,
 2479,
 5249,
 1570,
 2579,
 5129,
 3209,
 2282,
 2243,
 4487,
 1969,
 3305,
 261,
 3407,
 4047,
 4687,
 2520,
 5306,
 5117,
 289,
 3894,
 5467,
 3007,
 2330,
 3162,
 5330,
 4673,
 1453,
 5452,
 463,
 4353,
 3081,
 4627,
 2140,
 2788,
 5301,
 4317,
 2162,
 1730,
 3561,
 1820,
 4513,
 

In [587]:
# select edgeds within subgraph
edges = g.es.select(_within=index_feats_inds)

In [588]:
edges_igraph = [e.tuple for e in edges]

In [589]:
edges_igraph

[(2814, 4252),
 (2814, 4308),
 (2814, 4556),
 (2814, 2243),
 (2814, 4791),
 (2814, 2307),
 (2814, 1628),
 (2814, 4254),
 (2814, 2865),
 (2814, 5200),
 (2814, 4879),
 (2814, 925),
 (2814, 2450),
 (2814, 3325),
 (2814, 3519),
 (2814, 454),
 (2814, 4685),
 (2814, 866),
 (2814, 3166),
 (2814, 1459),
 (2814, 3902),
 (2814, 3126),
 (2814, 295),
 (2814, 1543),
 (2814, 4981),
 (2814, 1158),
 (2814, 2114),
 (2814, 4803),
 (2814, 699),
 (2814, 2433),
 (2814, 144),
 (2814, 1010),
 (2814, 362),
 (2814, 2082),
 (2814, 1595),
 (2814, 1657),
 (2814, 5402),
 (2814, 190),
 (2814, 791),
 (2814, 1818),
 (2814, 3909),
 (2814, 2582),
 (2814, 1894),
 (2814, 1935),
 (11, 4677),
 (11, 1934),
 (11, 3923),
 (11, 592),
 (11, 655),
 (11, 4317),
 (11, 3169),
 (11, 2613),
 (11, 3283),
 (11, 1343),
 (11, 5492),
 (11, 3501),
 (11, 3284),
 (11, 756),
 (11, 1058),
 (11, 4918),
 (11, 3586),
 (11, 411),
 (11, 2210),
 (11, 1321),
 (11, 2302),
 (16, 3007),
 (16, 284),
 (16, 3339),
 (16, 4857),
 (16, 5117),
 (16, 463),
 (16

In [590]:
# now we need to map them to row indices in the x feature matrix
# igraph_ind -> tw_ind -> row_ind

In [591]:
igraph_ind_to_tw_ind_map = {ig: tw for (tw, ig) in index_feats_inds_map.items()}

In [592]:
len(igraph_ind_to_tw_ind_map)

617

In [593]:
tw_ind_to_row_ind_map = {tw: i for (i, tw) in enumerate(index_feats)}

In [594]:
len(tw_ind_to_row_ind_map)

617

In [595]:
def map_ig_ind_to_row_ind(ig):
    return tw_ind_to_row_ind_map[igraph_ind_to_tw_ind_map[ig]]

In [596]:
def map_edge(e):
    a, b = e
    return (map_ig_ind_to_row_ind(a), map_ig_ind_to_row_ind(b))

In [597]:
edge_index = [map_edge(e) for e in edges_igraph]

In [598]:
len(edge_index)

22801

In [599]:
d = Data(x=x, edge_index=edge_index)

In [600]:
d

Data(x=[617, 7], edge_index=[22801])

In [601]:
def create_pyg_data_objects(tweet_ids, user_id, neighbour_users):
    '''
        Given tweets, central user and neighbour_users,
        we extract a pytorchgeometric Data representation
        of the 'neighbour activity' for each tweet
        
        nodes = user + retweeting neighbors + other neighbors
        features:
            centralities
            is_central_user
            is_retweeting
    '''
    s = open_session()
    user = s.query(User).get(user_id)

    ### Compute index mappings (igraph, twid, row_id)

    # print("# Filter centralities to cover only ngids")
    # This is the fixed order we will keep for nodes in x,
    # for all the generated datasamples for this central user
    user_id = user.id
    neighbour_ids = [u.id for u in neighbour_users]    
    index_users = [user] + neighbour_users
    index_twid = [user_id] + neighbour_ids    
    index_twid_to_igraph_map = {int(l): i for (i,l) in enumerate(g.vs["twid"]) if int(l) in index_twid}
    index_ig = [index_twid_to_igraph_map[l] for l in index_twid]
    index_centralities = [np.array(m)[index_ig] for m in centralities]

    ### Compute fixed centrality features
    centralities_matrix = np.vstack(index_centralities).transpose()

    is_central_col = np.zeros(len(index_users))
    is_central_col[0] = 1
    is_central_col = np.expand_dims(is_central_col, axis=1)

    ## Compute edges
    # select edgeds within subgraph
    edges = g.es.select(_within=index_ig)
    edges_igraph = [e.tuple for e in edges]

    # now we need to map them to row indices in the x feature matrix
    # igraph_ind -> tw_ind -> row_ind

    igraph_ind_to_tw_ind_map = {ig: tw for (tw, ig) in index_twid_to_igraph_map.items()}
    tw_ind_to_row_ind_map = {tw: i for (i, tw) in enumerate(index_twid)}

    edge_index = torch.tensor([map_edge(e) for e in edges_igraph])
    edge_index = edge_index.transpose(0,1)

    ### Compute retweeting features for each example
    data_objects = []
    for tweet_id in tweet_ids:
#         print(tweet_id)
        is_retweeting_col = np.zeros(len(index_users))
        for i, u in enumerate(index_users):
            if tweet_id in [t.id for t in u.timeline]:
                is_retweeting_col[i] = 1
        is_retweeting_col = np.expand_dims(is_retweeting_col, axis=1)

        x = np.hstack([centralities_matrix, is_central_col, is_retweeting_col])
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(int(tweet_id in [t.id for t in user.timeline]), dtype=torch.long)
        
        d = Data(x=x, edge_index=edge_index, y=y)
        data_objects.append(d)
        
    return data_objects

In [602]:
user_id

362040004

In [603]:
s = open_session()
user = s.query(User).get(user_id)
# remove central user from neighbours
neighbour_users = [u for u in get_level2_neighbours(user, s) if u.id != user.id]

In [604]:
tweet_ids = Xy_train.index

In [605]:
len(tweet_ids)

3500

In [606]:
data_objects = create_pyg_data_objects(tweet_ids, user_id, neighbour_users)

In [607]:
sum([d.y for d in data_objects])

tensor(19)

In [608]:
d = data_objects[5]

In [609]:
d.x

tensor([[1.1137e-04, 3.8729e+02, 3.9668e-01,  ..., 4.0000e+00, 1.0000e+00,
         0.0000e+00],
        [1.3649e-04, 7.0620e+02, 3.7754e-01,  ..., 4.0000e+00, 0.0000e+00,
         0.0000e+00],
        [7.1109e-04, 9.5859e+04, 4.6832e-01,  ..., 3.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [7.3185e-04, 5.4295e+04, 4.5361e-01,  ..., 4.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.8125e-04, 6.3580e+02, 3.6812e-01,  ..., 4.0000e+00, 0.0000e+00,
         0.0000e+00],
        [9.3056e-04, 6.8329e+04, 4.4579e-01,  ..., 4.0000e+00, 0.0000e+00,
         0.0000e+00]])

In [610]:
d.edge_index

tensor([[387, 387, 387,  ..., 288, 288, 288],
        [438, 582, 602,  ..., 542, 428, 447]])

In [611]:
d.edge_index.transpose(0,1)

tensor([[387, 438],
        [387, 582],
        [387, 602],
        ...,
        [288, 542],
        [288, 428],
        [288, 447]])

In [612]:
d.x.dtype

torch.float32

In [613]:
d.x.shape

torch.Size([617, 7])

In [614]:
len(edge_index)

22801

In [615]:
torch.tensor(d.x, dtype=torch.float16).dtype

/tmp/ipykernel_227926/3563974785.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(d.x, dtype=torch.float16).dtype


torch.float16

In [616]:
type(torch.tensor(d.x))

/tmp/ipykernel_227926/2985997979.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  type(torch.tensor(d.x))


torch.Tensor

In [617]:
torch.Tensor

torch.Tensor

In [618]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.0.1+cpu


# Graph Classification with Graph Neural Networks

[Previous: Node Classification with Graph Neural Networks](https://colab.research.google.com/drive/14OvFnAXggxB8vM4e8vSURUp1TaKnovzX)

In this tutorial session we will have a closer look at how to apply **Graph Neural Networks (GNNs) to the task of graph classification**.
Graph classification refers to the problem of classifiying entire graphs (in contrast to nodes), given a **dataset of graphs**, based on some structural graph properties.
Here, we want to embed entire graphs, and we want to embed those graphs in such a way so that they are linearly separable given a task at hand.

In [619]:
torch.manual_seed(12345)
# dataset = dataset.shuffle()

split = int(len(data_objects) * 0.7)
train_dataset = data_objects[:split]
test_dataset = data_objects[split:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 2450
Number of test graphs: 1050


In [620]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(x=[39488, 7], edge_index=[2, 1459264], y=[64], batch=[39488], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(x=[39488, 7], edge_index=[2, 1459264], y=[64], batch=[39488], ptr=[65])

Step 3:
Number of graphs in the current batch: 64
DataBatch(x=[39488, 7], edge_index=[2, 1459264], y=[64], batch=[39488], ptr=[65])

Step 4:
Number of graphs in the current batch: 64
DataBatch(x=[39488, 7], edge_index=[2, 1459264], y=[64], batch=[39488], ptr=[65])

Step 5:
Number of graphs in the current batch: 64
DataBatch(x=[39488, 7], edge_index=[2, 1459264], y=[64], batch=[39488], ptr=[65])

Step 6:
Number of graphs in the current batch: 64
DataBatch(x=[39488, 7], edge_index=[2, 1459264], y=[64], batch=[39488], ptr=[65])

Step 7:
Number of graphs in the current batch: 64
DataBatch(x=[39488, 7], edge_index=[2, 1459264], y=[64], batch=[39488], ptr=[65])

Step 8:
Number of graphs in the current batch: 64
DataBatch(x=[39488,

## Training a Graph Neural Network (GNN)

Training a GNN for graph classification usually follows a simple recipe:

1. Embed each node by performing multiple rounds of message passing
2. Aggregate node embeddings into a unified graph embedding (**readout layer**)
3. Train a final classifier on the graph embedding

There exists multiple **readout layers** in literature, but the most common one is to simply take the average of node embeddings:

$$
\mathbf{x}_{\mathcal{G}} = \frac{1}{|\mathcal{V}|} \sum_{v \in \mathcal{V}} \mathcal{x}^{(L)}_v
$$

PyTorch Geometric provides this functionality via [`torch_geometric.nn.global_mean_pool`](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.glob.global_mean_pool), which takes in the node embeddings of all nodes in the mini-batch and the assignment vector `batch` to compute a graph embedding of size `[batch_size, hidden_channels]` for each graph in the batch.

The final architecture for applying GNNs to the task of graph classification then looks as follows and allows for complete end-to-end training:

In [621]:
num_node_features = x.shape[1]
num_node_features

7

In [622]:
num_classes = 2

In [623]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index, batch):        
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=5)
print(model)

GCN(
  (conv1): GCNConv(7, 5)
  (conv2): GCNConv(5, 5)
  (conv3): GCNConv(5, 5)
  (lin): Linear(in_features=5, out_features=2, bias=True)
)


Here, we again make use of the [`GCNConv`](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv) with $\mathrm{ReLU}(x) = \max(x, 0)$ activation for obtaining localized node embeddings, before we apply our final classifier on top of a graph readout layer.

Let's train our network for a few epochs to see how well it performs on the training as well as test set:

In [624]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 171):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.9951, Test Acc: 0.9933
Epoch: 002, Train Acc: 0.9951, Test Acc: 0.9933


KeyboardInterrupt: 

As one can see, our model reaches around **76% test accuracy**.
Reasons for the fluctations in accuracy can be explained by the rather small dataset (only 38 test graphs), and usually disappear once one applies GNNs to larger datasets.



## (Optional) Exercise

Can we do better than this?
As multiple papers pointed out ([Xu et al. (2018)](https://arxiv.org/abs/1810.00826), [Morris et al. (2018)](https://arxiv.org/abs/1810.02244)), applying **neighborhood normalization decreases the expressivity of GNNs in distinguishing certain graph structures**.
An alternative formulation ([Morris et al. (2018)](https://arxiv.org/abs/1810.02244)) omits neighborhood normalization completely and adds a simple skip-connection to the GNN layer in order to preserve central node information:

$$
\mathbf{x}_v^{(\ell+1)} = \mathbf{W}^{(\ell + 1)}_1 \mathbf{x}_v^{(\ell)} + \mathbf{W}^{(\ell + 1)}_2 \sum_{w \in \mathcal{N}(v)} \mathbf{x}_w^{(\ell)}
$$

This layer is implemented under the name [`GraphConv`](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GraphConv) in PyTorch Geometric.

As an exercise, you are invited to complete the following code to the extent that it makes use of PyG's `GraphConv` rather than `GCNConv`.
This should bring you close to **82% test accuracy**.

In [ ]:
from torch_geometric.nn import GraphConv


class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GNN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GraphConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)        
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        x = global_mean_pool(x, batch)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GNN(hidden_channels=64)
print(model)

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GNN(hidden_channels=64)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, 201):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

## Conclusion

In this chapter, you have learned how to apply GNNs to the task of graph classification.
You have learned how graphs can be batched together for better GPU utilization, and how to apply readout layers for obtaining graph embeddings rather than node embeddings.

In the next session, you will learn how you can utilize PyTorch Geometric to let Graph Neural Networks scale to single large graphs.

[Next: Scaling Graph Neural Networks](https://colab.research.google.com/drive/1XAjcjRHrSR_ypCk_feIWFbcBKyT4Lirs)